In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 200)
pd.set_option('display.float_format', lambda x: f'{x:,.3f}')


In [2]:
PATH = "./data/"  # 네 로컬 경로에 맞게

app_train = pd.read_csv(PATH + "application_train.csv")
app_test  = pd.read_csv(PATH + "application_test.csv")

bureau = pd.read_csv(PATH + "bureau.csv")
bureau_balance = pd.read_csv(PATH + "bureau_balance.csv")

prev = pd.read_csv(PATH + "previous_application.csv")
pos = pd.read_csv(PATH + "POS_CASH_balance.csv")
cc = pd.read_csv(PATH + "credit_card_balance.csv")
inst = pd.read_csv(PATH + "installments_payments.csv")


In [3]:
tables = {
    "application_train": app_train,
    "application_test": app_test,
    "bureau": bureau,
    "bureau_balance": bureau_balance,
    "previous_application": prev,
    "POS_CASH": pos,
    "credit_card": cc,
    "installments": inst
}

summary = []

for name, df in tables.items():
    summary.append({
        "table": name,
        "rows": df.shape[0],
        "cols": df.shape[1]
    })

pd.DataFrame(summary)


,table,rows,cols
0,application_train,307511,122
1,application_test,48744,121
2,bureau,1716428,17
3,bureau_balance,27299925,3
4,previous_application,1670214,37
5,POS_CASH,10001358,8
6,credit_card,3840312,23
7,installments,13605401,8


In [6]:
app_train['TARGET'].value_counts(normalize=True)

TARGET
0   0.919
1   0.081
Name: proportion, dtype: float64

In [7]:
ext_cols = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
app_train[ext_cols].describe()


,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
count,"134,133.000","306,851.000","246,546.000"
mean,0.502,0.514,0.511
std,0.211,0.191,0.195
min,0.015,0.000,0.001
25%,0.334,0.392,0.371
50%,0.506,0.566,0.535
75%,0.675,0.664,0.669
max,0.963,0.855,0.896


In [9]:
app_train[ext_cols].isna().mean().sort_values(ascending=False)

EXT_SOURCE_1   0.564
EXT_SOURCE_3   0.198
EXT_SOURCE_2   0.002
dtype: float64

In [10]:
num_cols = app_train.select_dtypes(include=['int64', 'float64']).columns
cat_cols = app_train.select_dtypes(include=['object']).columns

len(num_cols), len(cat_cols)

(106, 16)

In [11]:
[c for c in num_cols if c.startswith("DAYS")]


['DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'DAYS_LAST_PHONE_CHANGE']

In [13]:
app_train[['DAYS_BIRTH', 'DAYS_EMPLOYED']].describe()

,DAYS_BIRTH,DAYS_EMPLOYED
count,"307,511.000","307,511.000"
mean,"-16,036.995","63,815.046"
std,"4,363.989","141,275.767"
min,"-25,229.000","-17,912.000"
25%,"-19,682.000","-2,760.000"
50%,"-15,750.000","-1,213.000"
75%,"-12,413.000",-289.000
max,"-7,489.000","365,243.000"


In [14]:
app_train.nunique().sort_values().head(10)

LIVE_CITY_NOT_WORK_CITY    2
FLAG_DOCUMENT_18           2
FLAG_DOCUMENT_14           2
FLAG_MOBIL                 2
FLAG_EMP_PHONE             2
FLAG_WORK_PHONE            2
FLAG_CONT_MOBILE           2
FLAG_PHONE                 2
FLAG_EMAIL                 2
FLAG_DOCUMENT_15           2
dtype: int64

In [16]:
missing = (
    app_train
    .isna()
    .mean()
    .sort_values(ascending=False)
)

missing.head(20)

COMMONAREA_MEDI            0.699
COMMONAREA_AVG             0.699
COMMONAREA_MODE            0.699
NONLIVINGAPARTMENTS_MODE   0.694
NONLIVINGAPARTMENTS_AVG    0.694
NONLIVINGAPARTMENTS_MEDI   0.694
FONDKAPREMONT_MODE         0.684
LIVINGAPARTMENTS_MODE      0.684
LIVINGAPARTMENTS_AVG       0.684
LIVINGAPARTMENTS_MEDI      0.684
FLOORSMIN_AVG              0.678
FLOORSMIN_MODE             0.678
FLOORSMIN_MEDI             0.678
YEARS_BUILD_MEDI           0.665
YEARS_BUILD_MODE           0.665
YEARS_BUILD_AVG            0.665
OWN_CAR_AGE                0.660
LANDAREA_MEDI              0.594
LANDAREA_MODE              0.594
LANDAREA_AVG               0.594
dtype: float64

In [17]:
app_train.groupby('TARGET')[ext_cols].mean()

,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
TARGET,,,
0,0.511,0.523,0.521
1,0.387,0.411,0.391


In [19]:
app_train.groupby('TARGET')['AMT_INCOME_TOTAL'].median()

TARGET
0   148,500.000
1   135,000.000
Name: AMT_INCOME_TOTAL, dtype: float64